In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')
english_stopwords = stopwords.words('english')
combined_stopwords = list(set(turkish_stopwords + english_stopwords))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ganga\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import pandas as pd
train_classification = pd.read_csv('train-classification.csv')
train_classification = train_classification.rename(columns={'Unnamed: 0': 'username', 'label': 'label'})

train_classification["label"] = train_classification['label'].apply(str.lower)
username_category = train_classification.set_index('username').to_dict()['label']

# Load the new CSV file
annotated_users = pd.read_csv('annotated_users_CS412-5189b0cc4cb9.csv')

for index, row in annotated_users.iterrows():
    if index == 0:  # If it is the first row, skip
        continue
    username = row['username']
    if pd.isnull(row['influencerCategory']):
        continue
    influencer_category = row['influencerCategory'].strip().lower()  # Normalize (remove spaces and convert to lowercase)
    if username not in username_category:
        username_category[username] = influencer_category

# Apply the same process when adding the other file
annotated_users = pd.read_csv('annotated_users_CS412-0e2ff92120f3.csv')
for index, row in annotated_users.iterrows():
    if index == 0:  # If it is the first row, skip
        continue
    username = row['username']
    if pd.isnull(row['influencerCategory']):
        continue
    influencer_category = row['influencerCategory'].strip().lower()  # Normalize
    if username not in username_category:
        username_category[username] = influencer_category

# Additionally, normalize all existing categories
username_category = {k: v.strip().lower() for k, v in username_category.items()}


In [4]:
normalized_table = {
    'mom and children' : "Mom and Children",
    "food" : "Food",
    'travel' : "Travel",
    'fashion' : "Fashion",
    'gaming' : "Gaming",
    'health and lifestyle' : "Health and Lifestyle",
    'entertainment' : "Entertainment",
    'tech': "Tech",
    'sports' : "Sports",
    'art' : "Art"
}
normalized_table

{'mom and children': 'Mom and Children',
 'food': 'Food',
 'travel': 'Travel',
 'fashion': 'Fashion',
 'gaming': 'Gaming',
 'health and lifestyle': 'Health and Lifestyle',
 'entertainment': 'Entertainment',
 'tech': 'Tech',
 'sports': 'Sports',
 'art': 'Art'}

In [5]:
annotated_users = pd.read_csv('annotated_users_CS412-5189b0cc4cb9.csv')
annotated_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   username            150 non-null    object
 1   url                 150 non-null    object
 2   influencerCategory  147 non-null    object
 3   influencerMention   147 non-null    object
 4   accountType         147 non-null    object
dtypes: object(5)
memory usage: 6.0+ KB


In [6]:
username_category

{'taskirancemal': 'mom and children',
 'tam_kararinda': 'food',
 'spart4nn': 'food',
 'sosyalyiyiciler': 'food',
 'sonaydizdarahad': 'mom and children',
 'somersivrioglu': 'food',
 'sinankoc': 'travel',
 'simulasyonturk': 'gaming',
 'savas_karakas_sudaki_izler': 'travel',
 'sakinenurunannesi': 'mom and children',
 'ruyabuyuktetik': 'fashion',
 'raykakumru': 'health and lifestyle',
 'pintipanda': 'gaming',
 'pinarindepresyonu': 'health and lifestyle',
 'pinarhotic': 'health and lifestyle',
 'pinardonmez_': 'mom and children',
 'ozgeninoltasi': 'food',
 'nayaozgun': 'fashion',
 'nataliyarcan': 'health and lifestyle',
 'muthispsikoloji': 'health and lifestyle',
 'murattekecicom': 'tech',
 'mrsisbeceren': 'mom and children',
 'mosyosokola': 'food',
 'mirandaorlayn': 'entertainment',
 'mezarci_pubg': 'gaming',
 'mervedemireltaskiran': 'travel',
 'lifewithapineapple': 'travel',
 'lalsefkatli': 'travel',
 'kutupanne': 'mom and children',
 'kosifcihan': 'health and lifestyle',
 'kisikatescom':

In [7]:
len(username_category)

2890

In [8]:
train_classification.groupby('label').count()

,username
label,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [9]:
import json

train_data_path = "training-dataset.jsonl"

username_post_train = dict()
username_profile_train = dict()

username_post_test = dict()
username_profile_test = dict()

# Reading the JSONL file
with open(train_data_path, "r", encoding="utf-8") as fh:
    for line in fh:
        sample = json.loads(line.strip())

        profile = sample["profile"]
        username = profile["username"]
        if username in username_category:  # Categorizing username based on category
            # Training data
            username_post_train[username] = sample["posts"]
            username_profile_train[username] = profile
        else:
            # Test data
            username_post_test[username] = sample["posts"]
            username_profile_test[username] = profile


In [10]:
train_profile = pd.DataFrame(username_profile_train).T.reset_index(drop=True)
test_profile = pd.DataFrame(username_profile_test).T.reset_index(drop=True)

train_profile.head(5)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,deparmedya,3170700063,Depar Medya,#mediaplanning #mediabuying #sosyalmedya,Local business,None,1167,192,True,False,...,None,None,LOCAL,False,False,https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,kafesfirin,266439571,KAFES FIRIN,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,Brand,None,11997,17,True,False,...,None,None,BRAND,False,False,https://instagram.fada1-13.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,vimerang,2367195567,Vimerang,Dijital İletişim Yönetimi🎬info@vimerang.comq,None,None,2321,454,True,False,...,Creators & Celebrities,None,VIDEO_CREATOR,False,False,https://instagram.fist19-1.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,yusufelibelediyesi08,43824173432,Yusufeli Belediye Başkanlığı,Yusufeli Belediye Başkanlığı Resmi İnstagram H...,Community,None,4635,83,True,False,...,None,None,None,False,False,https://instagram.fszf2-1.fna.fbcdn.net/v/t51....,False,False,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,mustafa_yalcinn38,9606564254,Mustafa Yalçın,Talas Belediye Başkanı,Politician,None,13647,29,True,False,...,None,None,POLITICIAN,False,False,https://instagram.fist1-4.fna.fbcdn.net/v/t51....,True,False,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [11]:
test_profile.head(5)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,konforyatak,8782109673,Konfor Yatak #KonforluUykular,"Konfor Yatak, birbirinden farklı özelliklere s...",Furniture,None,40334,2,True,False,...,None,None,FURNITURE,False,False,https://instagram.fyei6-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,ht_kulup,1950140344,HT KULÜP,Bloomberght - Habertürk Magazin,None,None,158877,69,True,False,...,None,None,TV_SHOW,False,False,https://instagram.fada2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,ajansspor,338611487,Ajansspor,"🏢 Saran Group \n🏟 Anında, tarafsız spor haberl...",News & media website,None,93193,286,True,False,...,None,None,NEWS_SITE,False,False,https://instagram.fadb2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [12]:
print(len(train_profile), len(test_profile))

2815 2600


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re


def preprocessing_text(text):
    text = text.casefold()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


corpus = []

training_usernames = []

for username,post in username_post_train.items():
    training_usernames.append(username)
    
    cleaned_captions = []
    for p in post:
        post_caption = p.get('caption', '')
        if post_caption is None:
            continue
        
        post_caption = preprocessing_text(post_caption)
        
        if post_caption != '':
            cleaned_captions.append(post_caption)

    user_post_captions = "\n".join(cleaned_captions)
    corpus.append(user_post_captions)



vectorizer = TfidfVectorizer(stop_words=combined_stopwords,max_features=7500)

vectorizer.fit(corpus)

x_post_train = vectorizer.transform(corpus)
y_train = [username_category[username] for username in training_usernames]

test_usernames = []
test_corpus = []

for username,post in username_post_test.items():
    test_usernames.append(username)
    
    cleaned_captions = []
    for p in post:
        post_caption = p.get('caption', '')
        if post_caption is None:
            continue
        
        post_caption = preprocessing_text(post_caption)
        
        if post_caption != '':
            cleaned_captions.append(post_caption)

    user_post_captions = "\n".join(cleaned_captions)
    test_corpus.append(user_post_captions)

x_post_test = vectorizer.transform(test_corpus)

In [14]:
assert y_train.count("NA") == 0

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ab', 'abd', 'abdulkadir', ..., 'şıklık', 'şıklığı', 'şımartın'],
      dtype=object)

In [16]:
len(feature_names)

7500

In [17]:
tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
tfidf.head()

,ab,abd,abdulkadir,abdullah,abi,abiye,abone,abs,academicistanbul,acar,...,şöleni,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şıklık,şıklığı,şımartın
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.044504,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.038686,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.044084,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [18]:
# give me where the most frequent words are used
tfidf.sum().sort_values(ascending=False)

bir          205.188603
olsun         63.131586
yeni          49.490676
istanbul      48.852084
olarak        46.134587
                ...    
soğanlık       0.364962
gökkuşaği      0.264777
yanyol         0.257858
tekira         0.196151
kivilcim       0.172342
Length: 7500, dtype: float64

In [19]:
tfidf['bir']

0       0.163253
1       0.216293
2       0.164618
3       0.051873
4       0.030249
          ...   
2810    0.020759
2811    0.027602
2812    0.012404
2813    0.018849
2814    0.174865
Name: bir, Length: 2815, dtype: float64

In [20]:
if "bir" in tfidf.columns:
    print("True")

True


In [21]:
filtered_version = tfidf.drop("bir",axis=1)
filtered_version

,ab,abd,abdulkadir,abdullah,abi,abiye,abone,abs,academicistanbul,acar,...,şöleni,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şıklık,şıklığı,şımartın
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.044504,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.038686,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.044084,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2810,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2811,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2812,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2813,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [22]:
if "bir" in filtered_version.columns:
    print("True")
else:
    print("False")

False


In [23]:
filtered_version.shape

(2815, 7499)

In [24]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

# Profile information of all users
train_profile = pd.DataFrame(username_profile_train).T

# Get category_name information
train_profile['category_name'] = train_profile['category_name'].str.split().str[0]
train_profile["category_name"] = train_profile["category_name"].fillna("unknown")  # Fill missing categories
encoder = OneHotEncoder()
category_features = encoder.fit_transform(train_profile["category_name"].values.reshape(-1, 1))

# Convert the TF-IDF matrix to a DataFrame
tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names, index=username_post_train.keys())

# Combine TF-IDF with category_name features
category_features_df = pd.DataFrame(category_features.toarray(), columns=encoder.get_feature_names_out(["category_name"]), index=train_profile.index)
combined_features = pd.concat([tfidf, category_features_df], axis=1)

# Ensure matching between `username_category` and `combined_features`
valid_usernames = [username for username in username_category.keys() if username in combined_features.index]
filtered_combined_features = combined_features.loc[valid_usernames]
filtered_y_train = [username_category[username] for username in valid_usernames]

# Check class distribution
class_counts = Counter(filtered_y_train)
print("Class Counts Before Handling:", class_counts)

# Select only classes with at least 2 examples
filtered_data = [(x, y) for x, y in zip(filtered_combined_features.index, filtered_y_train) if class_counts[y] > 1]
filtered_combined_features = filtered_combined_features.loc[[x for x, y in filtered_data]]
filtered_y_train = [y for x, y in filtered_data]

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(filtered_combined_features, filtered_y_train, test_size=0.2, stratify=filtered_y_train)

# Train using Naive Bayes
model = MultinomialNB()
model.fit(x_train, y_train)

# Performance evaluation
# Training data
y_train_pred = model.predict(x_train)
print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Train Classification Report:\n", classification_report(y_train, y_train_pred, zero_division=0))

# Validation data
y_val_pred = model.predict(x_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:\n", classification_report(y_val, y_val_pred, zero_division=0))


Class Counts Before Handling: Counter({'food': 518, 'health and lifestyle': 516, 'tech': 359, 'entertainment': 331, 'fashion': 305, 'travel': 295, 'art': 197, 'mom and children': 151, 'sports': 125, 'gaming': 17, '': 1})
Train Accuracy: 0.7347845402043536
Train Classification Report:
                       precision    recall  f1-score   support

                 art       0.97      0.36      0.52       157
       entertainment       0.62      0.63      0.62       265
             fashion       0.81      0.78      0.79       244
                food       0.83      0.95      0.89       414
              gaming       0.00      0.00      0.00        14
health and lifestyle       0.57      0.88      0.69       413
    mom and children       0.92      0.20      0.33       121
              sports       0.98      0.46      0.63       100
                tech       0.79      0.86      0.83       287
              travel       0.88      0.71      0.78       236

            accuracy          

In [25]:
test_data_path = "test-classification-round1.dat"

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

['ozhotelstr', 'elleturkiye', 'sozerinsaatorhangazi', 'sanliurfapiazzaavym', 'rusanozden']


In [26]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = training_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


test_unames.remove("screenname")

screenname


In [27]:
len(x_test)

999

In [28]:
len(test_unames)

999

In [29]:
import numpy as np

# List of all features used in training
training_feature_names = combined_features.columns

# Convert test data to a DataFrame
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

# Add missing columns and fill with 0
for col in training_feature_names:
    if col not in df_test.columns:
        df_test[col] = 0

# Arrange columns in the same order
df_test = df_test[training_feature_names]

# Make predictions
test_pred = model.predict(df_test)

# Generate output
output = dict()
for index, uname in enumerate(test_unames):  # Replace `test_unames` with `test_usernames`
    output[uname] = test_pred[index]

# Check the output
print(output)


C:\Users\ganga\AppData\Local\Temp\ipykernel_19212\2532582615.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[col] = 0
C:\Users\ganga\AppData\Local\Temp\ipykernel_19212\2532582615.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[col] = 0
C:\Users\ganga\AppData\Local\Temp\ipykernel_19212\2532582615.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis

{'ozhotelstr': 'travel', 'elleturkiye': 'fashion', 'sozerinsaatorhangazi': 'tech', 'sanliurfapiazzaavym': 'food', 'rusanozden': 'health and lifestyle', 'ikizlerbebefethiye': 'health and lifestyle', 'turkcelldehayat': 'health and lifestyle', 'zul.ays': 'fashion', 'tulinnalbayrak': 'food', 'turkesfilik': 'health and lifestyle', 'musteritemsilcisininicsesi': 'health and lifestyle', 'denizfeneriorg': 'health and lifestyle', 'gulgrupgayrimenkul': 'health and lifestyle', 'bridgerestaurant': 'food', 'icindekocvar': 'tech', 'zaimuniv': 'health and lifestyle', 'iskefeleather': 'tech', 'lekesizsarrafiye': 'fashion', 'kentfmtr': 'entertainment', 'gilletteturkiye': 'health and lifestyle', 'gupseo': 'health and lifestyle', 'lemonadeagency': 'tech', 'onikisubatbeltr': 'health and lifestyle', 'solaravm': 'tech', 'desimerkez': 'tech', 'melikesahin': 'entertainment', 'odabodrum': 'travel', 'planports': 'tech', 'imvu': 'fashion', 'refahist': 'health and lifestyle', 'moc_coffeeofficial': 'food', 'venusaj

In [30]:
len(df_test)

999

In [31]:
# Filter columns that start with 'category_name'
category_columns = df_test.columns[df_test.columns.str.startswith('category_name')]

# Find columns with values different from 0
non_zero_columns = category_columns[df_test[category_columns].ne(0).any()].tolist()

# Print the result
print(non_zero_columns)


[]


In [32]:
df_test.columns
category_columns = []
for col in df_test.columns:
    if col.startswith('category_name'):
        category_columns.append(col)

category_columns

['category_name_Actor',
 'category_name_Advertising',
 'category_name_Advertising/Marketing',
 'category_name_Aerospace',
 'category_name_Agricultural',
 'category_name_Agriculture',
 'category_name_Animal',
 'category_name_Apartment',
 'category_name_App',
 'category_name_Aquarium',
 'category_name_Architectural',
 'category_name_Art',
 'category_name_Artist',
 'category_name_Arts',
 'category_name_Athlete',
 'category_name_Author',
 'category_name_Automation',
 'category_name_Automotive',
 'category_name_Aviation',
 'category_name_Baby',
 'category_name_Bags/Luggage',
 'category_name_Bakery',
 'category_name_Bar',
 'category_name_Beach',
 'category_name_Beauty',
 'category_name_Beauty,',
 'category_name_Bicycle',
 'category_name_Biotechnology',
 'category_name_Blinds',
 'category_name_Blogger',
 'category_name_Boat',
 'category_name_Book',
 'category_name_Bookstore',
 'category_name_Brand',
 'category_name_Bridal',
 'category_name_Broadcasting',
 'category_name_Building',
 'category_

In [33]:
normalized_table

{'mom and children': 'Mom and Children',
 'food': 'Food',
 'travel': 'Travel',
 'fashion': 'Fashion',
 'gaming': 'Gaming',
 'health and lifestyle': 'Health and Lifestyle',
 'entertainment': 'Entertainment',
 'tech': 'Tech',
 'sports': 'Sports',
 'art': 'Art'}

In [34]:
test_pred = model.predict(df_test)

output_normalized = dict()

for index, uname in enumerate(test_unames):
    predicted_label = test_pred[index].strip().lower()  # Convert the predicted category to lowercase
    normalized_label = normalized_table.get(predicted_label, predicted_label.title())  # Normalize or default to title case
    output_normalized[uname] = normalized_label


In [35]:
len(test_pred)

999

In [36]:
with open("prediction-classification-round1.json", "w") as of:
  json.dump(output_normalized, of, indent=4)

In [37]:
# check for data inbalance
train_classification.groupby('label').count()

,username
label,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [38]:
# check for test prediction distribution
pd.Series(test_pred).value_counts()

health and lifestyle    459
food                    148
tech                    125
entertainment           109
travel                   75
fashion                  72
art                       4
sports                    4
mom and children          3
Name: count, dtype: int64

## Regression Part

In [39]:
from pprint import pprint

test_regression_datapath = "test-regression-round1.jsonl"

test_regression_data = []
with open(test_regression_datapath, "rt") as fh:
  for line in fh:
    test_regression_data.append(json.loads(line.strip()))

pprint(test_regression_data[15])

{'caption': '#Repost wine_surgeon with make_repost\n'
            '・・・\n'
            'Yedi Bilgeler’ den kütür kütür bir Emir geldi… VindemiaDefne '
            '2020…\n'
            'Yıllardır gözlemlerime göre Emir’ in yeri bir başkadır şarap '
            'severlerin gönlünde… Üzümün isminin çağrıştırdığı saf ve net Öz '
            'Türkçe tınısı gibi bir şarap yapmış 7 Bilgeler. Özlediğimiz saf '
            've net  Emir’ den kastım; yeşil tonların, güçlü mineralite ile '
            'harmanlandığı, eşlik eden güçlü asidite ve dengeli alkol ile '
            'damakta kütür kütür tazelik hissi veren bir karakter. Son '
            'yıllarda özellikle bölgesel üreticilerimizin de ilgisinin yeniden '
            'Emir’ e kaymasıyla keyifli şaraplarını deneyimleme şansını '
            'yakalamıştık. Yağlımsı, yuvarlak, tatlımsı olgun meyveli, bir '
            'Narince yada Chardonnay hatırlatan, göreceli olarak biraz hantal '
            'kalan örneklerdi. Ancak bu ‘Emir’ gerçekte

In [48]:
from collections import defaultdict

def predict_like_count(username, current_post=None):
    def get_avg_like_count_by_media_type(posts: list):
        media_type_likes = defaultdict(list)
        for post in posts:
            if current_post is not None and post["id"] == current_post["id"]:
                continue
            
            media_type = post.get("media_type", "UNKNOWN")
            like_count = post.get("like_count", 0)
            if like_count is None:
                like_count = 0
            media_type_likes[media_type].append(like_count)
        
        avg_likes_by_media_type = {}
        for media_type, likes in media_type_likes.items():
            if likes:
                avg_likes_by_media_type[media_type] = sum(likes) / len(likes)
            else:
                avg_likes_by_media_type[media_type] = 0.0
        
        return avg_likes_by_media_type

    if username in username_post_train:
        avg_likes = get_avg_like_count_by_media_type(username_post_train[username])
    elif username in username_post_test:
        avg_likes = get_avg_like_count_by_media_type(username_post_test[username])
    else:
        print(f"No data available for {username}")
        return -1

    if current_post and current_post.get("media_type") in avg_likes:
        return avg_likes[current_post["media_type"]]
    else:
        return sum(avg_likes.values()) / len(avg_likes) if avg_likes else 0.0


In [49]:
def log_mse_like_counts_log10(y_true, y_pred):
    """
    Calculate the Log Mean Squared Error (Log MSE) for like counts (log10(like_count + 1)).

    Parameters:
    - y_true: array-like, actual like counts
    - y_pred: array-like, predicted like counts

    Returns:
    - log_mse: float, Log Mean Squared Error
    """
    # Ensure inputs are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Log transformation: log10(like_count + 1)
    log_y_true = np.log10(y_true + 1)
    log_y_pred = np.log10(y_pred + 1)

    # Compute squared errors
    squared_errors = (log_y_true - log_y_pred) ** 2

    # Return the mean of squared errors
    return np.mean(squared_errors)


In [50]:
#@title Train Dataset evaluation

y_like_count_train_true = []
y_like_count_train_pred = []
for uname, posts in username_post_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train= {log_mse_like_counts_log10(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train= 0.22634931052566004


In [43]:

import json

path = "test-regression-round1.jsonl"
output_path = "prediction-regression-round1.json"

# Sonuçları bir sözlükte saklayacağız
output_dict = {}

with open(path, "rt") as fh:
    for line in fh:
        sample = json.loads(line)

        # Tahmin yap
        pred_val = predict_like_count(sample["username"])
        sample["like_count"] = int(pred_val)

        # ID'yi anahtar, tahmini değeri değer olarak ekle
        output_dict[sample["id"]] = sample["like_count"]
        

# JSON'u belirtilen formatta yaz
with open(output_path, "wt") as of:
    json.dump(output_dict, of, indent=4)


In [44]:
from pprint import pprint
pprint(output_dict)

{'17841711964015880': 543,
 '17842125969098625': 68,
 '17842262878347614': 41,
 '17842364658102051': 30,
 '17842536582014367': 37,
 '17842698372057811': 29,
 '17842876767045279': 81625,
 '17842964729217076': 50,
 '17843011057181888': 78,
 '17843101473063062': 35,
 '17843594979071884': 94,
 '17843911590088401': 68,
 '17844053652063317': 489,
 '17844197268085344': 237,
 '17844573736118484': 11,
 '17844869978048961': 3462,
 '17845290365889769': 39,
 '17845321360490579': 101,
 '17845574133093613': 55,
 '17845713315070947': 154,
 '17845819792530553': 11,
 '17846057727068087': 69,
 '17846418540053860': 44,
 '17846468333175606': 56582,
 '17846613890183781': 52,
 '17846868575978527': 26,
 '17847270349486793': 7,
 '17847275544045247': 27,
 '17847419060812430': 10621,
 '17847600461892145': 26,
 '17847651789079092': 72,
 '17848078085798050': 129,
 '17848377382859036': 19,
 '17848602387082437': 1597,
 '17848734028757169': 35,
 '17849004903051130': 2638,
 '17849359799737045': 74,
 '1784936070007135

In [45]:
my_test_df = pd.read_csv('annotated_users_CS412-5189b0cc4cb9.csv')
my_test_df

,username,url,influencerCategory,influencerMention,accountType
0,santrasosyal,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Health and Lifestyle,No,Company
1,sortiesortie,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Entertainment,No,Company
2,bodrumlupastalar,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Food,No,Company
3,ihealthsaglik,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Health and Lifestyle,No,Company
4,bisikletyolculugu,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Sports,Yes,Influencer
...,...,...,...,...,...
145,fringeistanbul,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Entertainment,No,Company
146,46oyunculari,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Art,No,Company
147,twitchdenkesitler,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Gaming,No,Influencer
148,stmdefence,https://docs.google.com/forms/d/e/1FAIpQLSfyzd...,Tech,No,Company


In [46]:
len(username_category)

2890

In [47]:
len(output_dict)

3000